#NOTEBOOK 5: Regresión
En este notebook, nos concentraremos en predecir la densidad del vino usando otro tipo de técnicas. Primero, encontraremos las correlaciones entre densidad y las otras variables. Después, usaremos la típica técnica de "Regresión lineal univariada" para predecir la densidad usando una única variable predictora. Por último, emplearemos varias técnicas de regresión multivariada de machine learning y las compararemos entre sí usando métricas de performance.



## Tabla de contenidos
---

* [Correlaciones](#Correlaciones)
* [Regresión Lineal](#Regresión_Lineal)
* [Regresión Lineal múltiple](#Regresión_Lineal_múltiple)
* [Escalado](#Escalado)
* [LASSO](#Lasso)




# Correlaciones
---

Para ajustar los datos con una modelo de regresión lineal, es buena práctica emeplar variables que presenten una alta correlación con el objetivo ("target") Dos maneras de hacer esto es calculando los coeficientes de correlación, o bien usando métodos visuales.

Predeciremos la densidad para vinos tintos (red whine), así que importaremos los datos y luego analizaremos el dataset.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", delimiter=";")

Podemos aplicar el método **corr** de Pandas para calcular las correlaciones entre pares para las columnas del dataset [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html 'pandas.DataFrame.corr')

In [ ]:
correlations = df.corr()['density'].drop(['quality', 'density'])
print(correlations)

Otra forma de hacerlo, es realizar un gráfico interactivo con ipywidgets que nos permita visualizar qué tan diferentes son las correlaciones.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

Predictor_Var = list(df.columns)

def CorrVis(Predictor_Var):
  plt.scatter(df['density'],df[Predictor_Var])
  plt.xlabel('Density')
  plt.ylabel(Predictor_Var)
  print('Corr Coef = ', np.corrcoef(df['density'],df[Predictor_Var])[0,1])

widgets.interactive(CorrVis, Predictor_Var = list(df.columns))

#Regresión Lineal
---

En esta sección, vamos a realizar una regresión lineal usando "alcohol" como variable predictora ($x_i$) y densidad como objetivo ($y_i$), de acuerdo al modelo:

$
y_i = β_1 x_i + β_0
$

Separaremos los datos en un set de entrenamiento un set de testeo, lo cual se utiliza habitualmente en métodos de machine learning para la validación del modelo creado. Para más información visita la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 'sklearn.model_selection.train_test_split')

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['alcohol']]
y = df['density']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Vamos a importar la librería para realizar regresión lineal y definir cual es la variable predictora y cual la variable objetivo. Para más información, visita la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html 'sklearn.linear_model.LinearRegression')

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X = X_train, y = y_train)

Imprimamos en la pantalla los parametros de la regresión lineal.

In [ ]:
print('β1 = ' + str(linear_regression.coef_) + ', β0 = ' + str(linear_regression.intercept_))

Podemos cuantificar qué tan bueno es el ajuste usando $R^2$. Para más información, visita la [documentación](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics 'Regression metrics') 

$
R^2 =1- \frac{\sum{ (y_i-\hat{y})^2}}{\sum{(y_i-\overline{y})^2}}
$

Los ajustes usualemnte son mejores en los sets de entrenamiento que en los sets de testeo, en este caso, encontramos que el set de entrenamiento tiene algunos outliers que hacen que el ajuste para set de entrenamiento no sea tan bueno.

In [ ]:
from sklearn.metrics import r2_score
y_pred_test = linear_regression.predict(X_test)
y_pred_train = linear_regression.predict(X_train)

print('R2 train = ', r2_score(y_train, y_pred_train))
print('R2 test = ', r2_score(y_test, y_pred_test))


Una manera de visualizar qué tan bueno es el ajuste es graficar el valor predicho contra el valor real.

In [ ]:
plt.scatter(y_train,y_pred_train, label='Training Set')
plt.scatter(y_test,y_pred_test, label='Test Set')
plt.xlabel('Real')
plt.ylabel('Predicted')
plt.legend()


#Regresión Lineal Múltiple
---

Regresión Lineal Múltiple (MLR) es una generalización de la clásica regresión lineal. MLR modela una regresión lineal entre la respuesta del objetivo y múltiples variable explicativas.
​	  
$y_i =β_0​	 +β_1	 x_{i1}​	 + β_2 x_{i2}​	 +...+ β_p​	 x_{ip}​
$

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['quality', 'density'], axis=1)
y = df['density']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Como MLR es una generalización, la librería de **Scikit Learn** usa la misma función que usamos antes.

In [ ]:
from sklearn.linear_model import LinearRegression

multiple_linear_regression = LinearRegression()
multiple_linear_regression.fit(X = X_train, y = y_train)

Aumentar el número de variables predictoras lleva a un mejor ajuste del objetivo, lo cual genera que el valor de $R^2$ incremente.

In [ ]:
from sklearn.metrics import r2_score

pred_train_lr = multiple_linear_regression.predict(X_train)
pred_test_lr = multiple_linear_regression.predict(X_test)

print('R2 training = ', r2_score(y_train, pred_train_lr))
print('R2 test = ', r2_score(y_test, pred_test_lr))

Otra métrica útil es $RSME$ y tiene la ventaja de que puede ser usada para modelos no lineales. Para más información, por favor visita la [documentación](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics 'Regression metrics')

$
RMSE =\sqrt{ \frac{1}{n} \sum{(y_i - \hat{y})^2}}
$

In [ ]:
from sklearn.metrics import mean_squared_error
rmse_test = np.sqrt(mean_squared_error(y_test,pred_test_lr))
print('RSME test= ', rmse_test)

Otra vez, podemos usar un gráfico para comparar el set de entrenamiento y el de testeo.

In [ ]:
plt.scatter(y_train,pred_train_lr, label='Training Set')
plt.scatter(y_test,pred_test_lr, label='Test Set')

plt.xlabel('Real')
plt.ylabel('Predicted')
plt.legend()

Analicemos los coeficientes de MLR y prestemos antención a su magnitud. Parece difícil poder concluir algo que acerca de su importancia.

In [ ]:
coeffecients = pd.DataFrame(multiple_linear_regression.coef_,X.columns.tolist())
coeffecients.columns = ['Coeficiente'] 
print(coeffecients)

In [ ]:
coeffecients.plot.bar()

#Estandarización
---

La estandarización es un método para transformas las diferentes variables predictoras en rangos comparable. Debido a que es un set de datos estandarizados (autoescalados), un coeficiente más alto indica que es una variable predictora que tiene mayor importancia para predecir el objetivo



In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['quality', 'density'], axis=1)
y = df['density']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Estandarizar después de dividir el dataset es una forma de evitar sesgo o error sistemático.


In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

multiple_linear_regression = LinearRegression()
multiple_linear_regression.fit(X = X_train, y = y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = multiple_linear_regression.predict(X_test)

rmse_MLR = np.sqrt(mean_squared_error(y_test, y_pred))

r2 = r2_score(y_test, y_pred)

print('R2 test = ', r2)
print('RSME test = ', rmse_MLR)

A medida que la data va siendo estandarizada, podemos analizar los coeficientes y determinar cuales son finalmente las variables predictores más importantes.

In [ ]:
coeffecients = pd.DataFrame(multiple_linear_regression.coef_,X.columns.tolist())
coeffecients.columns = ['Coeffecient'] 
print(coeffecients)

Encontramos que algunas variables no son tan importantes para generar el modelo de regresión. Vale la pena mencionar que usando solamente el "alcohol", "residual sugar", "fixed acidity", y el "pH", se podría hacer un mejor modelo ya que las otras variables no afectan notablemente la predicción e introducen ruido al modelo.

In [ ]:
coeffecients.plot.bar()

#LASSO
---

"Least Absolute Shrinkage and Selection Operator" - "Operador de Selección y Menor Encogimiento Absoluto" (LASSO) es un método de regresión lineal que produce una selección de variables y regularización para mejorar la precisión de la predicción y generar un modelo más pequeño. Este método usa una función de "costo", con una constante alfa que define el grado de penalización.

$
LASSO_{CostFunction}=\sum_{i=1}^M (y_i-\hat{y_i})^2=\sum_{i=1}^M (y_i-\sum_{j=0}^p w_j \times x_{ij})^2 + \alpha\sum_{j=0}^p |w_j| \\
For \; some \; t \, > \, 0, \, \sum_{j=0}^p |w_j|<t
$

**Scikit Learn** implementa LASSO en la función **sklearn.linear_model.Lasso**- Para más información, visita la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html 'sklearn.linear_model.Lasso')


In [ ]:
from sklearn.linear_model import Lasso

lasso_regression = Lasso(alpha=0.001)
lasso_regression.fit(X = X_train, y = y_train)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
y_pred = lasso_regression.predict(X_test)


r2 = r2_score(y_test, y_pred)
print('R2 test = ', r2)

Analizamos si usando LASSO tenemos un modelo más pequeño donde algunos de los coeficientes son cero.

In [ ]:
coeffecients = pd.DataFrame(lasso_regression.coef_,X.columns.tolist())
coeffecients.columns = ['Coeffecient'] 
print(coeffecients)

                      Coeffecient
fixed acidity            0.000518
volatile acidity         0.000000
citric acid              0.000000
residual sugar           0.001855
chlorides                0.000027
free sulfur dioxide     -0.000000
total sulfur dioxide     0.000090
pH                       0.000345
sulphates                0.000096
alcohol                 -0.001337


In [ ]:
coeffecients.plot.bar()

Analizaremos como el RSME de los sets de entrenamiento y testeo cambian para diferentes valores de alfa.

In [ ]:
alphas=np.logspace(-10,3,endpoint=True,num=100,base=10)
RMSE=[]
RMSE_p=[]
for x in (alphas):
    #print(x)
    model_lasso = Lasso(x)
    model_lasso.fit(X_train, y_train)
    pred_test_lasso= model_lasso.predict(X_test)
    pred_train_lasso=model_lasso.predict(X_train)
    RMSE_p.append(np.sqrt(mean_squared_error(y_test,pred_test_lasso)))
    RMSE.append(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
    

plt.plot(alphas,RMSE, label='Training Set')
plt.plot(alphas,RMSE_p, label='Test Set')
plt.plot(alphas,len(alphas)*[rmse_MLR], label='MLR')
plt.xscale("log")
plt.xlabel('alpha')
plt.ylabel('RMSE')
plt.legend()

Podemos ahora hacer un gráfico avanzado usando ipywidgets que nos permita cambiar los coeficientes de acuerdo a los valores de alfa.

In [ ]:
import ipywidgets as widgets

from IPython.display import display
from sklearn import preprocessing


data_lasso = df.copy()
data_lasso = data_lasso.drop(labels = ['quality','density'],axis = 1)
features = data_lasso.columns.tolist()
data_lasso = preprocessing.StandardScaler().fit_transform(data_lasso)
y_lasso = df['density']

def Lassovis(alpha):
    lasso_regression = Lasso((alpha))
    lasso_regression.fit(X = data_lasso, y = y_lasso)
        
    plt.figure(figsize=(10, 6))
    plt.subplot(211)
    pred_test_lasso=lasso_regression.predict(X_test)
    plt.scatter(y_test,pred_test_lasso)
    plt.xlabel('y_test')
    plt.ylabel('pred_test_lasso')

    plt.subplot(212)
    plt.bar(features,lasso_regression.coef_)
    plt.xticks(rotation=90)


widgets.interact(Lassovis,alpha=widgets.FloatLogSlider(name='Alpha', base=10, min=-5, max=-2, step=0.25, value=0.001));